In [2]:
import db.knowhere_db as kdb
import pipeline.pipeline as pipeline
import pandas as pd
import numpy as np
import pickle

In [3]:
reader = kdb.Reader('knowhere')

In [7]:
glen_H_data_raw = reader.get_dataframe_pivoted(collection='iphone', username='glen', commute=True,\
                                               min_date='2017-3-24', max_date='2017-03-25')

emil_H_data_raw = reader.get_dataframe_pivoted(collection='iphone', username='emil', commute=True,\
                                               min_date='2017-3-20 00:00:00', max_date='2017-03-25 00:00:00')

In [8]:
emil_H_data_raw

,Microphone Left Channel Level,Magnetometer y,Magnetometer x,Microphone Right Channel Level,Gravity z,Gravity y,Gravity x,Gyrometer y,Magnetometer z,Altimeter (Barometer) Pressure,...,Acceleration x,Acceleration y,Gyrometer z,Altimeter (Barometer) Relative Altitude,Gyrometer x,GPS Horizontal Accuracy,GPS Longitude,GPS Vertical Accuracy,GPS Latitude,GPS Altitude
2017-03-20 23:40:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.088715,...,NaN,NaN,NaN,-0.199463,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-20 23:40:34,NaN,NaN,NaN,NaN,0.046783,0.996235,0.072993,NaN,NaN,101.089912,...,0.101872,-0.061161,NaN,-0.299696,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-20 23:40:35,NaN,30.747101,25.380737,NaN,NaN,NaN,NaN,NaN,-386.609375,101.095932,...,NaN,NaN,NaN,-0.801352,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-20 23:40:36,-8.899610,NaN,NaN,-8.899610,NaN,NaN,NaN,0.216615,NaN,101.096352,...,NaN,NaN,-1.516016,-0.836609,-0.366937,65.000000,-73.992867,10.000000,40.752888,18.396904
2017-03-20 23:40:37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.096352,...,NaN,NaN,NaN,-0.836609,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-20 23:40:38,NaN,NaN,NaN,NaN,-0.030460,0.984251,0.174134,NaN,NaN,101.091888,...,-0.039445,0.018862,NaN,-0.464399,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-20 23:40:39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.091644,...,NaN,NaN,NaN,-0.443748,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-20 23:40:40,NaN,30.866211,29.055862,NaN,NaN,NaN,NaN,NaN,-401.304901,101.090363,...,NaN,NaN,NaN,-0.336969,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-20 23:40:41,-5.525160,NaN,NaN,-5.525160,NaN,NaN,NaN,-0.100746,NaN,101.090363,...,NaN,NaN,0.932177,-0.336969,0.965102,NaN,NaN,NaN,NaN,NaN
2017-03-20 23:40:42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.092476,...,NaN,NaN,NaN,-0.513256,NaN,65.000000,-73.992933,10.000000,40.752795,18.871239


In [4]:
glen_H_data = glen_H_data_raw.dropna(subset=['Acceleration z', 'Acceleration x', 'Acceleration y'], how='all')
emil_H_data = emil_H_data_raw.dropna(subset=['Acceleration z', 'Acceleration x', 'Acceleration y'], how='all')

In [5]:
glen_commute_labels = pickle.load( open("glen_commute_labels.p", "rb" ))

emil_commute_labels = pickle.load( open("emil_commute_labels.p", "rb" ))

In [6]:
class CommuteLabeler(object):
    
    def __init__(self, username, events, min_date, max_date):
        self.username = username
        self.events = events
        self.min_date = min_date
        self.max_date = max_date
        self.df = reader.get_dataframe_pivoted(collection='iphone_test3', username=username, commute=True,\
                                               min_date=min_date, max_date=max_date)
    def add_classifications(self):    
        '''Pass events as a list of tuples (datetime, eventcode) to classify different events'''
        self.df.index = pd.to_datetime(self.df.index)
        self.df.fillna(method='ffill', inplace = True)
        for days_events in self.events.iterkeys():
            if pd.to_datetime(days_events) in pd.date_range(start=self.min_date, end=self.max_date, freq='D'):
                for time, action in self.events[days_events]:
                    self.df.loc[days_events + " " + time, 'classification'] = action
        self.df.fillna(method='ffill', inplace = True)
        self.df.drop_duplicates(subset=['Acceleration z', 'Acceleration x', 'Acceleration y'], inplace = True)
        return self.df

    def label_raw_iphone(self):
        self.df['classification'] = None
        self.add_classifications()
        return self.df
    
    def write_labels_to_files(self):
        label_raw_iphone()
        self.df.to_csv("Labeled_" + self.username + "_" + self.min_date + " to " + self.max_date + ".csv")
        return "Finished!"

In [20]:
g = CommuteLabeler('glen', glen_commute_labels, '2017-03-19', '2017-03-25')
g_long_label = g.label_raw_iphone()

In [21]:
#Use this cell if you want to simplify your labels
g_long_label = g_long_label.replace(to_replace='T_E', value='T_S')
g_long_label = g_long_label.replace(to_replace='E_U', value='E')
g_long_label = g_long_label.replace(to_replace='E_D', value='E')
g_long_label = g_long_label.replace(to_replace='S_U', value='W')
g_long_label = g_long_label.replace(to_replace='S_D', value='W')
g_long_label = g_long_label.replace(to_replace='T_T', value='T_S')
g_long_label = g_long_label.replace(to_replace='T_B', value='T_D')
g_long_label = g_long_label.replace(to_replace='P_W', value='W')
g_long_label = g_long_label.replace(to_replace='P_S', value='S')


g_long_label.groupby("classification").size()

classification
D      1257
E        67
S       455
T_D    2404
T_S     519
W      2310
dtype: int64

In [35]:
g_long_label = g_long_label.loc['2017-03-19':'2017-03-23',:]

In [37]:
g_long_label.to_csv('Labeled_glen_2017-03-19_to_2017-03-23_simplified.csv')

In [12]:
#glen_as_a_class = CommuteLabeler('glen', glen_commute_labels, '03-24-2017', '03-25-2017')
emil_as_a_class = CommuteLabeler('emil', emil_commute_labels, '2017-3-20 00:00:00', '2017-3-25 00:00:00')

In [13]:
#glen_as_a_class.label_raw_iphone()
emil_as_a_class.label_raw_iphone()

,Microphone Left Channel Level,Magnetometer y,Magnetometer x,Microphone Right Channel Level,Gravity z,Gravity y,Gravity x,Gyrometer y,Magnetometer z,Altimeter (Barometer) Pressure,...,Acceleration y,Gyrometer z,Altimeter (Barometer) Relative Altitude,Gyrometer x,GPS Horizontal Accuracy,GPS Longitude,GPS Vertical Accuracy,GPS Latitude,GPS Altitude,classification
2017-03-20 23:40:34,NaN,NaN,NaN,NaN,0.046783,0.996235,0.072993,NaN,NaN,101.089912,...,-0.061161,NaN,-0.299696,NaN,NaN,NaN,NaN,NaN,NaN,None
2017-03-20 23:40:38,NaN,NaN,NaN,NaN,-0.030460,0.984251,0.174134,NaN,NaN,101.091888,...,0.018862,NaN,-0.464399,NaN,NaN,NaN,NaN,NaN,NaN,None
2017-03-20 23:40:43,NaN,NaN,NaN,NaN,0.264729,0.893759,0.362097,NaN,NaN,101.093124,...,0.164744,NaN,-0.567652,NaN,NaN,NaN,NaN,NaN,NaN,None
2017-03-20 23:40:47,NaN,NaN,NaN,NaN,-0.180229,0.936915,0.299513,NaN,NaN,101.087227,...,-0.041331,NaN,-0.075556,NaN,NaN,NaN,NaN,NaN,NaN,None
2017-03-20 23:40:52,NaN,NaN,NaN,NaN,0.110324,0.973839,0.198663,NaN,NaN,101.089417,...,-0.007438,NaN,-0.257891,NaN,NaN,NaN,NaN,NaN,NaN,None
2017-03-20 23:40:56,-29.93400,NaN,NaN,-29.93400,0.003506,0.982984,0.183657,-0.005260,NaN,101.087471,...,-0.003065,-0.018108,-0.095703,0.006449,65.000000,-73.992763,10.000000,40.752945,18.855745,None
2017-03-20 23:41:01,-29.47250,NaN,NaN,-29.47250,-0.024607,0.981236,0.191233,-0.031339,NaN,101.090790,...,0.003383,-0.004091,-0.372730,0.032766,65.000000,-73.992763,10.000000,40.752945,18.855745,None
2017-03-20 23:41:05,-29.47250,23.091721,47.498093,-29.47250,-0.112676,0.967823,0.224996,-0.031339,-383.490875,101.090240,...,0.017727,-0.004091,-0.326895,0.032766,65.000000,-73.992763,10.000000,40.752945,18.855745,None
2017-03-20 23:41:10,-29.47250,21.829346,48.846817,-29.47250,-0.011328,0.948655,0.316110,-0.031339,-381.773346,101.088875,...,-0.011384,-0.004091,-0.213062,0.032766,65.000000,-73.992785,10.000000,40.752982,18.757204,None
2017-03-20 23:41:14,-29.47250,21.829346,48.846817,-29.47250,0.011984,0.956028,0.293029,-0.031339,-381.773346,101.095695,...,-0.018101,-0.004091,-0.781710,0.032766,65.000000,-73.992785,10.000000,40.752982,18.757204,None


In [14]:
#glen_as_a_class.df.groupby("classification").size()
emil_as_a_class.df.groupby("classification").size()

classification
E_D     4
E_U     4
S       8
T_D    39
T_S    39
W      13
dtype: int64

In [15]:
#glen_as_a_class.df.to_csv('Labeled_Test_Data_Glen_03_25_2017.csv')
emil_as_a_class.df.to_csv('Labeled_Test_Data_Emil_03_25_2017.csv')